In [ ]:
#pip install httplib2

In [ ]:
import httplib2

In [18]:
import re
import pybel
from pybel.io.jupyter import to_jupyter
from utils import *
import seaborn as sns
from kg_gen_3 import *

In [ ]:
itmp_chem = pd.read_csv('C:\\Users\\reagon.karki\\Documents\\ITMP\\EOSC Future\\ITMP ChEMBL data\\CHEMBL4303805.csv',sep=';',usecols=['ChEMBL ID'])

In [ ]:
itmp_chem = list(itmp_chem['ChEMBL ID'])

In [ ]:
len(itmp_chem)

In [ ]:
itmp_dis = RetDrugInd(itmp_chem)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/itmp_drugInd.pkl')
# outfile = open(filename,'wb')
# pickle.dump(itmp_dis,outfile)

In [ ]:
itmp_mech = RetMech(itmp_chem)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/itmp_mech.pkl')
# outfile = open(filename,'wb')
# pickle.dump(itmp_mech,outfile)

In [ ]:
itmp_act = RetAct_pchem_5(itmp_chem)

In [ ]:
len(itmp_act)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/itmp_act.pkl')
# outfile = open(filename,'wb')
# pickle.dump(itmp_act,outfile)

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/itmp_mech.pkl','rb')
itmp_mech = pickle.load(infile)
infile.close()

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/itmp_drugInd.pkl','rb')
itmp_dis = pickle.load(infile)
infile.close()

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/itmp_act.pkl','rb')
itmp_act = pickle.load(infile)
infile.close()

In [ ]:
prot_actMech = Ret_chembl_protein(itmp_act) + Ret_chembl_protein(itmp_mech)
prot_actMech = set(prot_actMech)
prot_actMech = list(prot_actMech)

In [ ]:
actMech2uprot = chembl2uniprot(prot_actMech)

In [ ]:
uprot = Ret_uprotid(actMech2uprot)

In [ ]:
uprot_ext = ExtractFromUniProt(uprot)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/uprot.pkl')
# outfile = open(filename,'wb')
# pickle.dump(uprot_ext,outfile)

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/uprot.pkl','rb')
uprot_ext = pickle.load(infile)
infile.close()

In [ ]:
itmp_act

In [ ]:
itmp_act = chembl2gene2path(actMech2uprot,itmp_act)
itmp_mech = chembl2gene2path(actMech2uprot,itmp_mech)

In [ ]:
itmp_graph = pybel.BELGraph(name='ITMP Graph 2024', version="0.0.1")

In [ ]:
itmp_graph = chem2moa_rel(itmp_mech,'HGNC',itmp_graph)
itmp_graph = chem2dis_rel(itmp_dis,itmp_graph)
itmp_graph = chem2act_rel(itmp_act,'HGNC',itmp_graph)
itmp_graph = gene2path_rel(actMech2uprot,'HGNC',itmp_graph)
itmp_graph = uniprot_rel(uprot_ext,'HGNC',itmp_graph)

In [ ]:
advEvent = GetAdverseEvents(itmp_chem)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/sideEffects.pkl')
# outfile = open(filename,'wb')
# pickle.dump(advEvent,outfile)

In [ ]:
itmp_graph = chembl2adverseEffect_rel(advEvent,itmp_graph)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/itmpGraph.pkl')
# outfile = open(filename,'wb')
# pickle.dump(itmp_graph,outfile)

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/covid19_kg_2024.pkl','rb')
covidkg = pickle.load(infile)
infile.close()

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/itmpGraph.pkl','rb')
itmp_graph = pickle.load(infile)
infile.close()

In [ ]:
covidkg.summarize

In [ ]:
itmp_graph.summarize

In [ ]:
cvd_ppi = pd.read_csv('data/kgs/viral_diseases/covid19_kg_2024/covid_ppi.txt',sep='\t')
len(cvd_ppi)

In [ ]:
cvd_ppi.columns

In [ ]:
cvd_ppi.head(50)

In [ ]:
cvd_ppi = cvd_ppi[cvd_ppi['# ID(s) interactor A'].str.contains("uniprotkb")]
cvd_ppi = cvd_ppi[cvd_ppi['ID(s) interactor B'].str.contains("uniprotkb")]
cvd_ppi = cvd_ppi[cvd_ppi['Taxid interactor A'].str.contains("9606|2697049")]
cvd_ppi = cvd_ppi[cvd_ppi['Taxid interactor B'].str.contains("9606|2697049")]

int_a = cvd_ppi['# ID(s) interactor A']
temp = []
for item in int_a:
    prot = item.split(':')[1]
    #print(prot)
    temp.append(prot)
    #break
    
int_b = cvd_ppi['ID(s) interactor B']
temp_2 = []
for item in int_b:
    prot = item.split(':')[1]
    #print(prot)
    temp_2.append(prot)
    
cvd_ppi['Int_A'] = temp
cvd_ppi['Int_B'] = temp_2

In [ ]:
int_a = cvd_ppi['# ID(s) interactor A']
temp = []
for item in int_a:
    prot = item.split(':')[1]
    #print(prot)
    temp.append(prot)
    #break
    
int_b = cvd_ppi['ID(s) interactor B']
temp_2 = []
for item in int_b:
    prot = item.split(':')[1]
    #print(prot)
    temp_2.append(prot)

In [ ]:
cvd_ppi['Int_A'] = temp
cvd_ppi['Int_B'] = temp_2
len(cvd_ppi)

In [ ]:
temp = []
for tax in cvd_ppi['Taxid interactor A']:
    if '9606' in tax:
        temp.append('HGNC')
    else:
        temp.append('VP')
        
temp_2 = []
for tax in cvd_ppi['Taxid interactor B']:
    if '9606' in tax:
        temp_2.append('HGNC')
    else:
        temp_2.append('VP')
        
cvd_ppi['Species_A'] = temp
cvd_ppi['Species_B'] = temp_2

cvd_ppi = cvd_ppi.reset_index(drop=True)

In [ ]:
len(cvd_ppi)

In [ ]:
cvd_ppi.head()

In [ ]:
xx = ExtractFromUniProt(list(set(cvd_ppi['Int_A'])))

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/Ext_uprot_Int_A.pkl')
# outfile = open(filename,'wb')
# pickle.dump(xx,outfile)

In [ ]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/Ext_uprot_Int_A.pkl','rb')
xx = pickle.load(infile)
infile.close()

In [ ]:
xx

In [ ]:
xx['Q53F19']['Gene']

In [ ]:
cvd_ppi['Int_A']

In [ ]:
temp = []
for uprot in cvd_ppi['Int_A']:
    
    getProt = ExtractFromUniProt([uprot])
    if getProt[uprot]['Gene']:
        temp.append(getProt[uprot]['Gene'])
    else:
        temp.append(uprot)    
        

In [ ]:
cvd_ppi['Gene_A'] = temp

In [ ]:
temp = []
for uprot in cvd_ppi['Int_B']:
    
    getProt = ExtractFromUniProt([uprot])
    if getProt[uprot]['Gene']:
        temp.append(getProt[uprot]['Gene'])
    else:
        temp.append(uprot)    
        

In [ ]:
len(temp)

In [ ]:
cvd_ppi['Gene_B'] = temp
        

In [ ]:
def check_response(response):
    try:
        #print(response.raise_for_status())
        response.raise_for_status()
    except requests.HTTPError:
        print(response.json())
        raise
        
POLLING_INTERVAL = 5
def check_id_mapping_results_ready(job_id):
    while True:
        
        API_URL = "https://rest.uniprot.org"
        session = requests.Session()
        request = session.get(f"{API_URL}/idmapping/status/{job_id}")
        check_response(request)
        j = request.json()
        if "jobStatus" in j:
            if j["jobStatus"] == "RUNNING":
                print(f"Retrying in {POLLING_INTERVAL}s")
                time.sleep(POLLING_INTERVAL)
            else:
                raise Exception(j["jobStatus"])
        else:
            return bool(j["results"] or j["failedIds"])
        
def get_id_mapping_results_link(job_id):
    API_URL = "https://rest.uniprot.org"
    session = requests.Session()
    url = f"{API_URL}/idmapping/details/{job_id}"
    request = session.get(url)
    return(request)
    check_response(request)
    return request.json()["redirectURL"]

In [ ]:
def prot2crossRefDB(prot_list):
    
    import requests
    
    final_list = []
    
    for prot in tqdm(prot_list,desc='Fetching Cross-referenced DB ids'):
        
        #print(prot)
        temp_list = []
        
        temp_list.append(prot)
        
        files = {
        'from': (None, 'Gene_Name'),
        'to': (None, 'UniProtKB-Swiss-Prot'),
        'ids': (None, prot),
        'taxId': (None, '9606'),
        }
    
        response = requests.post('https://rest.uniprot.org/idmapping/run', files=files)
        
        response = response.json()
        
        job_id = response['jobId']
        
        #print(job_id)
        
#         try:
#             output = get_id_mapping_results_link(job_id)
            
#         except: 
#             output = get_id_mapping_results_link(job_id)
        
        API_URL = "https://rest.uniprot.org"
        session = requests.Session()
        url = f"{API_URL}/idmapping/details/{job_id}"
        
        check_job = check_id_mapping_results_ready(job_id)
        #print(check_job)
        
        
        request = session.get(url)
        
        #time.sleep(3)
        check_response(request)
        #print(check_response)
        #print(request.json())

        output= request.json()["redirectURL"]
        

        response = requests.get(output)
        
        response = response.json()
        
        if response['results']: 
        
        #print(response)
        
            uprot = response['results'][0]['to']['primaryAccession']

            temp_list.append(uprot)

            uniProtkbId = response['results'][0]['to']['uniProtkbId'] 
            organism = response['results'][0]['to']['organism']['scientificName']

            #get all crossRef DBs
            crossRefs = response['results'][0]['to']['uniProtKBCrossReferences']

            db_list = []
            for item in crossRefs:

                db_list.append(item['database'])


            if 'OpenTargets' in db_list:
                getIndex = db_list.index('OpenTargets')
                temp_list.append(crossRefs[getIndex]['id'])

            else:
                temp_list.append('None')


            if 'ChEMBL' in db_list:
                getIndex = db_list.index('ChEMBL')
                temp_list.append(crossRefs[getIndex]['id'])

            else:
                temp_list.append('None')


            if 'HGNC' in db_list:
                getIndex = db_list.index('HGNC')
                temp_list.append(crossRefs[getIndex]['id'])

            else:
                temp_list.append('None')

            temp_list.append(uniProtkbId)
            temp_list.append(organism)

            #print(temp_list)

            final_list.append(temp_list)
        
        #print(final_list)
        
    cols = ['Protein','UniProt','ENSG','ChEMBL','HGNC','uniProtkbId','Organism']
    
    df= pd.DataFrame(final_list,columns=cols)             
        
    return(df)
    

In [ ]:
aa = get_id_mapping_results_link('59be157c25c1815efedf04e3561e0b6335de7add')


In [ ]:
gene_9606 = pd.read_excel('data/misc/uniprotkb_taxonomy_id_9606_AND_reviewed_2024_06_20.xlsx')

In [ ]:
gene_9606.head()

In [ ]:
gene_list = ['IGFBP3',
 'IL13RA2',
 'MAGEA4',
 'LGALS1',
 'CA9',
 'CD74',
 'S100A6',
 'CD44',
 'LINC01029',
 'FN1',
 'TGFBI',
 'PAGE5',
 'ANXA1',
 'SLC14A1',
 'GPRC5A',
 'S100A16',
 'ITGB4',
 'TM4SF1',
 'LAMB1',
 'PRAME',
 'PRRX1',
 'TGFA',
 'ERBB3',
 'LTBR',
 'CAVIN1',
 'MAGEC2',
 'CSPG4',
 'LINC01419',
 'FOSL1']

In [ ]:
genes = list(gene_9606['Gene Names'])
genes = [x for x in genes if str(x) != 'nan']

In [ ]:
#xx = prot2crossRefDB(list(cvd_ppi['Int_A']))
#xx = prot2crossRefDB(['TNF','SNCA'])
xx = prot2crossRefDB(genes)

In [ ]:
xx

In [ ]:
cvd_ppi.to_csv('data/kgs/viral_diseases/covid19_kg_2024/cvd_ppi.csv')

In [ ]:
merged_kg = nx.compose(itmp_graph,covidkg)

In [ ]:
# filename = ('data/kgs/viral_diseases/covid19_kg_2024/covidKG_itmpChem_Graph.pkl')
# outfile = open(filename,'wb')
# pickle.dump(merged_kg,outfile)

In [4]:
infile = open('data/kgs/viral_diseases/covid19_kg_2024/covidKG_itmpChem_Graph.pkl','rb')
merged_kg = pickle.load(infile)
infile.close()

In [5]:
def getProtfromKG(mainGraph):

    prot_list = []
    for u, v, data in tqdm(mainGraph.edges(data=True),desc='Filtering Proteins/Genes'):
        
        if 'VP' in u.namespace:
            if u.name not in prot_list:
                prot_list.append(u.name)

        if 'VP' in v.namespace:
            if v.name not in prot_list:
                prot_list.append(v.name)
                
    return(prot_list)

In [6]:
vp = getProtfromKG(merged_kg)

Filtering Proteins/Genes:   0%|          | 0/438112 [00:00<?, ?it/s]

In [7]:
vp

['rep',
 'N',
 'P0DTC1',
 'S',
 'P0DTC3',
 'E',
 'P0DTC5',
 'P0DTC8',
 'P0DTD2',
 'P0DTC6',
 'P0DTC7',
 'P0DTF1']

In [8]:
ppi_final = pd.read_csv('data/kgs/viral_diseases/covid19_kg_2024/ppi_final.csv',index_col=0)

In [9]:
score = ppi_final['Confidence value(s)']
temp = []
# for item in score:
    
#     x = float(item.split(':')[1])
    
temp = [float(x.split(':')[1]) for x in score ]    

ppi_final['score'] = temp

In [10]:
ppi_final.head()

,# ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,...,Source database(s),Interaction identifier(s),Confidence value(s),Int_A,Int_B,Species_A,Species_B,Int_A_Gene,Int_B_Gene,score
0,uniprotkb:Q53F19,uniprotkb:P50395,uniprotkb:Q9NXZ4|uniprotkb:B3KWG7|uniprotkb:Q9...,uniprotkb:Q5SX88|uniprotkb:O43928|uniprotkb:Q9...,psi-mi:ncbp3_human(display_short)|psi-mi:Nucle...,psi-mi:gdib_human(display_short)|psi-mi:Rab GD...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623276|imex:IM-26480-1,intact-miscore:0.35,Q53F19,P50395,HGNC,HGNC,NCBP3,GDI2,0.35
1,uniprotkb:Q53F19,uniprotkb:P51148,uniprotkb:Q9NXZ4|uniprotkb:B3KWG7|uniprotkb:Q9...,intact:EBI-21355794|intact:EBI-2798296|uniprot...,psi-mi:ncbp3_human(display_short)|psi-mi:Nucle...,psi-mi:rab5c_human(display_short)|psi-mi:Ras-r...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623276|imex:IM-26480-1,intact-miscore:0.35,Q53F19,P51148,HGNC,HGNC,NCBP3,RAB5C,0.35
2,uniprotkb:Q53F19,uniprotkb:Q6I9Y2,uniprotkb:Q9NXZ4|uniprotkb:B3KWG7|uniprotkb:Q9...,uniprotkb:Q6P1L3|uniprotkb:Q8WUF2|uniprotkb:Q9...,psi-mi:ncbp3_human(display_short)|psi-mi:Nucle...,psi-mi:thoc7_human(display_short)|psi-mi:THO c...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623276|imex:IM-26480-1,intact-miscore:0.64,Q53F19,Q6I9Y2,HGNC,HGNC,NCBP3,THOC7,0.64
3,uniprotkb:Q53F19,uniprotkb:P38919,uniprotkb:Q9NXZ4|uniprotkb:B3KWG7|uniprotkb:Q9...,intact:EBI-21354615|uniprotkb:Q15033|uniprotkb...,psi-mi:ncbp3_human(display_short)|psi-mi:Nucle...,psi-mi:if4a3_human(display_short)|psi-mi:Eukar...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623276|imex:IM-26480-1,intact-miscore:0.53,Q53F19,P38919,HGNC,HGNC,NCBP3,EIF4A3,0.53
4,uniprotkb:Q53F19,uniprotkb:Q96FV9,uniprotkb:Q9NXZ4|uniprotkb:B3KWG7|uniprotkb:Q9...,intact:EBI-28984554|uniprotkb:B2RBP6|uniprotkb...,psi-mi:ncbp3_human(display_short)|psi-mi:Nucle...,psi-mi:thoc1_human(display_short)|psi-mi:THO c...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623276|imex:IM-26480-1,intact-miscore:0.64,Q53F19,Q96FV9,HGNC,HGNC,NCBP3,THOC1,0.64


In [21]:
ppi_final = ppi_final[ppi_final['score'] >= 0.8]
ppi_final = ppi_final.reset_index(drop=True)

In [22]:
ppi_final

,# ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,...,Source database(s),Interaction identifier(s),Confidence value(s),Int_A,Int_B,Species_A,Species_B,Int_A_Gene,Int_B_Gene,score
0,uniprotkb:P52298,uniprotkb:Q09161,uniprotkb:E9PAR5|uniprotkb:Q14924|uniprotkb:Q2...,uniprotkb:Q59G76|uniprotkb:Q5T1V0|uniprotkb:B2...,psi-mi:ncbp2_human(display_short)|psi-mi:Nucle...,psi-mi:ncbp1_human(display_short)|psi-mi:Nucle...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623424|imex:IM-26480-7,intact-miscore:0.96,P52298,Q09161,HGNC,HGNC,NCBP2,NCBP1,0.96
1,uniprotkb:P52298,uniprotkb:Q09161,uniprotkb:E9PAR5|uniprotkb:Q14924|uniprotkb:Q2...,uniprotkb:Q59G76|uniprotkb:Q5T1V0|uniprotkb:B2...,psi-mi:ncbp2_human(display_short)|psi-mi:Nucle...,psi-mi:ncbp1_human(display_short)|psi-mi:Nucle...,"psi-mi:""MI:0096""(pull down)",Gebhardt A. et al.(2015),intact:EBI-20623177|pubmed:26382858|imex:IM-26480,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-20623690|imex:IM-26480-3,intact-miscore:0.96,P52298,Q09161,HGNC,HGNC,NCBP2,NCBP1,0.96
2,uniprotkb:Q14155,uniprotkb:Q14160,intact:EBI-25643109|uniprotkb:B1ALK6|uniprotkb...,uniprotkb:Q6P496|uniprotkb:Q7Z5D1|uniprotkb:Q8...,psi-mi:arhg7_human(display_short)|psi-mi:Rho g...,psi-mi:scrib_human(display_short)|psi-mi:Prote...,"psi-mi:""MI:0084""(phage display)",Ivarsson Y. et al.(2014),intact:EBI-11668009|pubmed:24550280|imex:IM-25074,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0471""(MINT, Dpt of Biology, Univers...",intact:EBI-11791066|imex:IM-25074-2,intact-miscore:0.94,Q14155,Q14160,HGNC,HGNC,ARHGEF7,SCRIB,0.94
3,uniprotkb:Q14160,uniprotkb:Q14155,uniprotkb:Q6P496|uniprotkb:Q7Z5D1|uniprotkb:Q8...,intact:EBI-25643109|uniprotkb:B1ALK6|uniprotkb...,psi-mi:scrib_human(display_short)|psi-mi:Prote...,psi-mi:arhg7_human(display_short)|psi-mi:Rho g...,"psi-mi:""MI:0084""(phage display)",Ivarsson Y. et al.(2014),intact:EBI-11668009|pubmed:24550280|imex:IM-25074,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0471""(MINT, Dpt of Biology, Univers...",intact:EBI-11791897|imex:IM-25074-150,intact-miscore:0.94,Q14160,Q14155,HGNC,HGNC,SCRIB,ARHGEF7,0.94
4,uniprotkb:Q14160,uniprotkb:Q14155,uniprotkb:Q6P496|uniprotkb:Q7Z5D1|uniprotkb:Q8...,intact:EBI-25643109|uniprotkb:B1ALK6|uniprotkb...,psi-mi:scrib_human(display_short)|psi-mi:Prote...,psi-mi:arhg7_human(display_short)|psi-mi:Rho g...,"psi-mi:""MI:0084""(phage display)",Ivarsson Y. et al.(2014),intact:EBI-11668009|pubmed:24550280|imex:IM-25074,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0471""(MINT, Dpt of Biology, Univers...",intact:EBI-11791933|imex:IM-25074-90,intact-miscore:0.94,Q14160,Q14155,HGNC,HGNC,SCRIB,ARHGEF7,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,uniprotkb:O94826,uniprotkb:P0DTD2,uniprotkb:D3DN48|ensembl:ENSP00000284320.5|int...,intact:EBI-25475909|uniprotkb:P0DTD2,psi-mi:tom70_human(display_short)|psi-mi:Mitoc...,psi-mi:orf9b_sars2(display_short)|psi-mi:ORF9b...,"psi-mi:""MI:0096""(pull down)",Brandherm L. et al.(2021),intact:EBI-27088033|pubmed:34502139|imex:IM-29236,taxid:9606(human)|taxid:9606(Homo sapiens),...,"psi-mi:""MI:0469""(European Bioinformatics Insti...",intact:EBI-27091279|imex:IM-29236-1,intact-miscore:0.98,O94826,P0DTD2,HGNC,VP,TOMM70,9b,0.98
651,uniprotkb:O94826,uniprotkb:P0DTD2,uniprotkb:D3DN48|ensembl:ENSP00000284320.5|int...,intact:EBI-25475909|uniprotkb:P0DTD2,psi-mi:tom70_human(display_short)|psi-mi:Mitoc...,psi-mi:orf9b_sars2(display_short)|psi-mi:ORF9b...,"psi-mi:""MI:0096""(pull down)",Brandherm L. et al.(2021),intact:EB

In [23]:
ppi_kg = pybel.BELGraph(name='COVID19 PPIs', version="0.0.1")

In [ ]:
bb = 'intact:EBI-20623177|pubmed:26382858|imex:IM-26480'

import re

#Check if the string starts with "The" and ends with "Spain":

evidence = re.search("pubmed:[0-9]+", bb)
x = x.group()
x


In [24]:
def ppi_rel(ppi_df,
    proteinList,
    graph: BELGraph
) -> BELGraph:
    """Method to add PPI relations to the KG

    :param PPIs with 2 columns as proteins and 2 columns as organism abbreviation i.e. HGNC 
    :param Protein list from KG
    :param graph:
    :return:
    """
    
    
    for prot_a, prot_b, uprot_a, uprot_b, sp_a, sp_b, pubmed in zip(ppi_df['Int_A_Gene'],ppi_df['Int_B_Gene'],ppi_df['Int_A'],
                             ppi_df['Int_B'],ppi_df['Species_A'],ppi_df['Species_B'],ppi_df['Publication Identifier(s)']):
        
        
        #regex to extract pubmedid 
    
        evidence = re.search("pubmed:[0-9]*", pubmed)
        evidence = evidence.group()
        #print(evidence)
        
        #condition 1
        if sp_a == 'HGNC' and sp_b == 'HGNC':
    
            graph.add_association(
            Protein(namespace=sp_a, name=prot_a),
            Protein(namespace=sp_b, name=prot_b),
            citation = 'IntAct',
            evidence = evidence
            )
        
        #condition 2
        if sp_a == 'HGNC' and sp_b == 'VP':
            
            if prot_b in proteinList:
    
                graph.add_association(
                Protein(namespace=sp_a, name=prot_a),
                Protein(namespace=sp_b, name=prot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
            if prot_b not in proteinList:
                
                graph.add_association(
                Protein(namespace=sp_a, name=prot_a),
                Protein(namespace=sp_b, name=uprot_b),
                citation = 'IntAct',
                evidence = evidence
                )
          
        #condition 3
        if sp_a == 'VP' and sp_b == 'HGNC':
            
            if prot_a in proteinList:
    
                graph.add_association(
                Protein(namespace=sp_a, name=prot_a),
                Protein(namespace=sp_b, name=prot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
            if prot_a not in proteinList:
                
                graph.add_association(
                Protein(namespace=sp_a, name=uprot_a),
                Protein(namespace=sp_b, name=prot_b),
                citation = 'IntAct',
                evidence = evidence
                )
    
        #condition 4
        if sp_a == 'VP' and sp_b == 'VP':
            
            if prot_a in proteinList and prot_b in proteinList:

                graph.add_association(
                Protein(namespace=sp_a, name=prot_a),
                Protein(namespace=sp_b, name=prot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
            if prot_a not in proteinList and prot_b not in proteinList:

                graph.add_association(
                Protein(namespace=sp_a, name=uprot_a),
                Protein(namespace=sp_b, name=uprot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
                        
            if prot_a in proteinList and prot_b not in proteinList:

                graph.add_association(
                Protein(namespace=sp_a, name=prot_a),
                Protein(namespace=sp_b, name=uprot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
            if prot_a not in proteinList and prot_b  in proteinList:

                graph.add_association(
                Protein(namespace=sp_a, name=uprot_a),
                Protein(namespace=sp_b, name=prot_b),
                citation = 'IntAct',
                evidence = evidence
                )
            
    return graph

In [25]:
xx = ppi_rel(ppi_final,vp,ppi_kg)

In [26]:
to_jupyter(xx)

<IPython.core.display.Javascript object>

In [ ]:
#to cytoscape compatible graphml 
pybel.to_graphml(merged_kg,'data/kgs/viral_diseases/covid19_kg_2024/covidKG_itmpChem_Graph.graphml')


In [ ]:
merged_kg.summarize

In [ ]:
node_data = {'Node':['Pathology','BiologicalProcess','Abundance','Protein'],
       'Number':[9005,5669,19931,1347]}
node = pd.DataFrame(node_data)
a = sns.barplot(x="Node", y="Number", data=node)
a.set(xlabel='Node',ylabel='Number',title= 'KG nodes in numbers (Total=35952)')

In [ ]:
nspace_data = {'Namespace':['ChEMBLAssay','ChEMBL','GOBP','GOMF','HGNC','Disease','SideEffect','Reactome','MOA'],
       'Number':[15835,4096,2085,1332,1347,1585,7420,1513,739]}
nspace = pd.DataFrame(nspace_data)
a = sns.barplot(x="Number", y="Namespace", data=nspace_data)
a.set(xlabel='Number',ylabel='Namespace',title= 'KG Namespace in numbers')

In [ ]:
#to cytoscape compatible graphml 
pybel.to_graphml(itmp_graph,'data/export//bycovidgraph_2ndSep.graphml')

#to regular BEL format
pybel.dump(itmp_graph,'data/export//bycovidgraph_2ndSep.bel')

#to neo4j
pybel.to_csv(itmp_graph,'data/export//bycovidgraph_2ndSep.csv')

#to sif
pybel.to_sif(itmp_graph,'data/export//bycovidgraph_2ndSep.sif')

In [ ]:
chemblprot = pd.read_csv('data/misc/chembl_uniprot_mapping.txt',sep='\t')

In [ ]:
chemblprot